In [2]:

from collections import namedtuple
%load_ext autoreload
%autoreload 2

In [3]:

from farmbase_agent_toolkit.api import FarmbaseAPI

api = FarmbaseAPI(secret_key="fdsfds", context=None)

get_tool_defs([api.create_farm, api.create_field, api.do_nothing])

[{'type': 'function',
  'function': {'name': 'create_farm',
   'description': 'This tool creates a new farm in Farmbase',
   'parameters': {'properties': {'name': {'type': 'string'},
     'location': {'type': 'string'}},
    'required': ['name', 'location'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'create_field',
   'description': 'This tool creates a new field in Farmbase',
   'parameters': {'$defs': {'LineString': {'description': 'LineString Model',
      'properties': {'bbox': {'anyOf': [{'maxItems': 4,
          'minItems': 4,
          'prefixItems': [{'type': 'number'},
           {'type': 'number'},
           {'type': 'number'},
           {'type': 'number'}],
          'type': 'array'},
         {'maxItems': 6,
          'minItems': 6,
          'prefixItems': [{'type': 'number'},
           {'type': 'number'},
           {'type': 'number'},
           {'type': 'number'},
           {'type': 'number'},
           {'type': 'number'}],
          'ty

In [2]:
import inspect


def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "name": func.__name__,
        "description": (func.__doc__ or "").strip(),
        "parameters": {
            "type": "object",
            "properties": parameters,
            "required": required,
        },
    }

In [6]:
from pprint import pprint
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

load_dotenv(find_dotenv())


# Example usage
def some_function(
        parameter1: int,  # Some description
        parameter2: tuple[int, int] = (1, 2),  # p2 description
):
    """
    some_function docstring
    """
    pass


def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    I can tell you about the black boot
    """
    print("Hello, world")


# schema =  function_to_schema(sample_function)


tools = [sample_function]
tool_schemas = [function_to_schema(tool) for tool in tools]

pprint(tool_schemas)

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Look up the black boot."}],
    # input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tool_schemas
)

# print(response.output)
message = response.choices[0].message

message.tool_calls[0].function
# strict=True : BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'some_function': In context=('properties', 'parameter2'), 'minItems' is not permitted.", 'type': 'invalid_request_error', 'param': 'tools[0].parameters', 'code': 'invalid_function_parameters'}}

# strict=False : BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'some_function': In context=('properties', 'parameter2'), array schema missing items.", 'type': 'invalid_request_error', 'param': 'tools[0].parameters', 'code': 'invalid_function_parameters'}}

[{'function': {'description': 'I can tell you about the black boot',
               'name': 'sample_function',
               'parameters': {'properties': {'param_1': {'type': 'string'},
                                             'param_2': {'type': 'string'},
                                             'some_optional': {'type': 'string'},
                                             'the_third_one': {'type': 'integer'}},
                              'required': ['param_1',
                                           'param_2',
                                           'the_third_one'],
                              'type': 'object'}},
  'type': 'function'}]


Function(arguments='{"param_1":"black","param_2":"boot","the_third_one":1}', name='sample_function')

In [8]:
from openai import OpenAI

client = OpenAI()

tools = [{'description': 'gets the count of animals and size of the farm for a user',
          'name': 'count_and_size',
          'parameters': {'$defs': {'User': {'additionalProperties': False,
                                            'properties': {'age': {'title': 'Age',
                                                                   'type': 'integer'},
                                                           'name': {'title': 'Name',
                                                                    'type': 'string'}},
                                            'required': ['name', 'age'],
                                            'title': 'User',
                                            'type': 'object'}},
                         'additionalProperties': False,
                         'properties': {'count': {'title': 'Count', 'type': 'integer'},
                                        'size': {'anyOf': [{'type': 'number'},
                                                           {'type': 'null'}],
                                                 'title': 'Size'},
                                        'user': {'$ref': '#/$defs/User'}},
                         'required': ['count', 'size', 'user'],
                         'title': 'simple_method_ParameterModel',
                         'type': 'object'},
          'strict': True,
          'type': 'function'}]

response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "Get the count of animals for user Peter?"}],
    tools=tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"count":0,"size":null,"user":{"age":0,"name":"Peter"}}', call_id='call_ngGmjwCYTQ4kH5wy8pQ5C55P', name='count_and_size', type='function_call', id='fc_67dd57dcc5d081928deddad7162ca8ce0e2824a3081bcd99', status='completed')]


In [27]:


tools = [{'description': 'This tool creates a new field in Farmbase\n',
  'name': 'create_field',
  'parameters': {'$defs': {'Position2D': {'items': {'type': 'number'},
                                          'type': 'array'}},
                 'additionalProperties': False,
                 'properties': {'boundary': {'description': 'the boundary of '
                                                            'the field as a '
                                                            'list of [long, '
                                                            'lat] coordinates.',
                                             'items': {'$ref': '#/$defs/Position2D'},
                                             'title': 'Boundary',
                                             'type': 'array'},
                                'farm_id': {'description': 'The ID of the farm '
                                                           'that the field '
                                                           'belongs to.',
                                            'title': 'Farm Id',
                                            'type': 'string'},
                                'name': {'description': 'the name of the '
                                                        'field.',
                                         'title': 'Name',
                                         'type': 'string'}},
                 'required': ['farm_id', 'name', 'boundary'],
                 'title': 'create_field_ParameterModel',
                 'type': 'object'},
  'strict': True,
  'type': 'function'}]

response = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "register my new field"}],
    tools=tools
)

print(response.output)

[ResponseOutputMessage(id='msg_67dd6918c3988192b42399145a311c7700f46f408b974f40', content=[ResponseOutputText(annotations=[], text="Could you please provide the details for the new field? I'll need the following information:\n\n1. **Name** of the field.\n2. **Boundary** coordinates (a list of `[longitude, latitude]` pairs defining the field's boundary).\n3. **Farm ID** that the field belongs to.", type='output_text')], role='assistant', status='completed', type='message')]


In [17]:
import asyncio
from collections import namedtuple
from dotenv import load_dotenv

load_dotenv("/Users/markns/workspace/farmwise/.env")

from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: tuple[float, float]
    temperature_range: str
    conditions: str


LatLong = namedtuple('LatLong', "lat long")

@function_tool
def get_weather(lat_lon: tuple[float, float]) -> Weather:
    print("[debug] get_weather called")
    return Weather(city=lat_long, temperature_range="14-20C", conditions="Sunny with wind.")


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


await main()

Error getting response: Error code: 400 - {'error': {'message': "Invalid schema for function 'get_weather': In context=('properties', 'lat_lon'), 'minItems' is not permitted.", 'type': 'invalid_request_error', 'param': 'tools[0].parameters', 'code': 'invalid_function_parameters'}}. (request_id: req_ae8166f28d94d39f86ae7cc7dc686d96)


BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'get_weather': In context=('properties', 'lat_lon'), 'minItems' is not permitted.", 'type': 'invalid_request_error', 'param': 'tools[0].parameters', 'code': 'invalid_function_parameters'}}